In [81]:
# IMPORTING LIBRARIES AND DEFINE IMPORT FUNCTION

import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from scipy import signal
from scipy.optimize import curve_fit
import plotly.graph_objects as go
from sound_card_cal import sc_cal_coeff as C_interp


def importWAV(filename):
    samplerate, rawData = wavfile.read(filename)
    
    time = np.linspace(0, rawData.shape[0]/samplerate, rawData.shape[0])   
    
    data = {'left':rawData[:, 0],'right':rawData[:, 1]}
    return time,data

In [82]:
# Amplifier Code

# Resistor values (Ohm)
R_1 = 100
R_2 = 20E3
R_3 = 47E3
R_4 = 200
R_5 = 2E3

# Capacitor values (F)
C_1 = 680E-12
C_2 = 0.22E-6
C_3 = 1E-9

def Z_c(C, f):
    """
    Capacitor impedance
    """
    
    return 1 / (2 * np.pi * f * C * complex(0, 1))


def VDG(Z_1, Z_2):
    """
    Gain factor for voltage divider Op-amp circuit
    """

    return (Z_1 + Z_2) / Z_1


def VDF(Z_1, Z_2):
    """
    Voltage divider circuit gain factor
    """

    return Z_1 / (Z_2 + Z_1)


def c_amp(f):
    """
    Final complex amplification factor for entire circuit
    """
    
    return VDG(R_1, 1/(1/R_2 + 1/Z_c(C_1, f))) * VDF(R_3, Z_c(C_2, f)) * VDG(R_4, 1/(1/R_5 + 1/Z_c(C_3, f)))


def amplification(c_amp):
    """
    Converts complex amplification factor to real amplification factor
    """
    
    return np.sqrt(c_amp * c_amp.conjugate())

def phase(c_amp):
    """
    Converts complex amplification factor to phase shift
    """

    # c_amp lies in Q1 of complex plane
    if c_amp.real >= 0 and c_amp.imag >= 0:
        return np.arctan(c_amp.imag / c_amp.real)
    
    # c_amp lies in Q2 of complex plane
    elif c_amp.real < 0 and c_amp.imag >= 0:
        return -np.pi + np.arctan(c_amp.imag / c_amp.real)
   
    # c_amp lies in Q3 of complex plane
    elif c_amp.real < 0 and c_amp.imag < 0:
        return np.arctan(c_amp.imag / c_amp.real) - np.pi

    # c_amp lies in Q4 of complex plane
    else:
        return np.arctan(c_amp.imag / c_amp.real)

V_1 = 50E-3
cR_1 = 120.26E3
cf_arr = np.array([])

for i in range(21):
    f = 10 ** (i * 0.25)
    ecR_2 = cR_1 / (amplification(c_amp(f)) / 2 - 1)
    cf_arr = np.append(cf_arr, round(f))

def v_into_amp(V_in, R_1, R_2):
    return V_in * R_2 / (R_1 + R_2)


cR_2 = np.array([982.4, 982.4, 391.017, 391.017, 200.64, 150.241, 150.241, 120.543, 120.543,
                 120.543, 120.543, 120.543, 120.543, 120.543, 120.543, 120.543, 120.543,
                 120.543, 325.049, 392.225, 980.54])
obs_V_out = np.array([0.0512/2, 0.117/2, 0.0888/2, 0.227/2, 0.188/2, 0.206/2, 0.249/2, 0.217/2, 
                      0.228/2, 0.224/2, 0.223/2, 0.22/2, 0.22/2, 0.218/2, 0.211/2, 0.191/2,
                      0.153/2, 0.101/2, 0.158/2, 0.091/2, 0.0925/2])

V_in = v_into_amp(V_1, cR_1, cR_2)

G = obs_V_out / V_in

def fourier_series(x, a0, an, bn, T):
    series = a0  # Start with the constant term
    for n in range(1, len(an) + 1):
        series += an[n - 1] * np.cos(2 * np.pi * n * np.log10(x) / T) + bn[n - 1] * np.sin(2 * np.pi * n * np.log10(x) / T)
    return series

T = 5

fourier_coeffs = np.fft.fft(G) / len(cf_arr)

a0 = fourier_coeffs[0].real
an = 2 * fourier_coeffs[1:len(fourier_coeffs) // 2].real
bn = -2 * fourier_coeffs[1:len(fourier_coeffs) // 2].imag

def mod_fourier_series(x, a, b, c, d, e, f, g, h, j):
    return fourier_series(x, a0, an, bn, T) + a + b*(x+c) + d*(x+e)**2 + f*(x+g)**3 + h*(x+j)**4


params, cov = curve_fit(mod_fourier_series, cf_arr, G)

def G_piece_func(x):
    y = np.array([])
    
    for i in x:
        if i < 10:
            y = np.append(y, fourier_series(i, a0, an, bn, T) - 11)
        elif i < 56:
            y = np.append(y, mod_fourier_series(i, *params))
        elif i < 70:
            y = np.append(y, mod_fourier_series(i, *params) * 2.3 - 2840)
        elif i < 1000:
            y = np.append(y, fourier_series(i, a0, an, bn, T))
        elif i < 2500:
            y = np.append(y, amplification(c_amp(i)))
        elif i < 4000:
            y = np.append(y, amplification(c_amp(i)) * 1.2 - 440)
        elif i < 20000:
            y = np.append(y, mod_fourier_series(i, *params) + 10) 
        else:
            y = np.append(y, amplification(c_amp(i))*0.9 - 30)
    return y.real

In [83]:
# Noise data import for test plot

# time,data = importWAV('1k_Ohm.wav')

# v_data = {'left': [], 'right': []}

# v_data['left'] = data['left'] / 8.695
# v_data['right'] = data['right'] / 8.695

# # plt.figure()
# # plt.plot(time, v_data['left'], label='left')
# # plt.plot(time, v_data['right'], label='right')
# # plt.xlabel('Time (s)')
# # plt.ylabel('Voltage (V)')
# # plt.xlim(0,0.002)
# # plt.title('Amplified noise from resistor at room temp.')

# # plt.legend()

In [84]:
# Code for test PSD

# fs = 48000

# rf, rPxx_den = signal.welch(v_data['right'], fs, nperseg=1024)
# lf, lPxx_den = signal.welch(v_data['left'], fs, nperseg=1024)

# rPxx_den = rPxx_den / G_piece_func(rf) ** 2
# lPxx_den = lPxx_den / G_piece_func(lf) ** 2

# noisepowerden = np.average(rPxx_den[5:400]) # Make sure this does not include elements beyond the length of Pxx_den
# print("Average noise power density over the stated range is", noisepowerden)
# print(len(rPxx_den))

# fig1, (ax1, ax2) = plt.subplots(2)

# ax1.set_xlabel('Frequency [Hz]', fontsize=10)
# ax1.set_ylabel('PSD [V**2/Hz]', fontsize=10)
# ax1.set_title('Power Spectral Density, linear scale', fontsize=14)
# ax2.set_xlabel('Frequency [Hz]', fontsize=10)
# ax2.set_ylabel('PSD [V**2/Hz]', fontsize=10)
# ax2.set_title('Power Spectral Density, semilog scale', fontsize=14)
# ax1.plot(rf, rPxx_den)
# ax2.semilogy(rf, rPxx_den)
# fig1.tight_layout()

In [85]:
# Importing voltage noise data and resistance values

time18, data18_1 = importWAV('Noise Signals/18_Ohm1.wav')
time56, data56_1 = importWAV('Noise Signals/56_Ohm1.wav')
time120, data120_1 = importWAV('Noise Signals/120_Ohm1.wav')
time300, data300_1 = importWAV('Noise Signals/300_Ohm1.wav')
time1k, data1k_1 = importWAV('Noise Signals/1k_Ohm1.wav')
time2k, data2k_1 = importWAV('Noise Signals/2k_Ohm1.wav')
time4k, data4k_1 = importWAV('Noise Signals/4k_Ohm1.wav')
time7k, data7k_1 = importWAV('Noise Signals/7k_Ohm1.wav')
time12k, data12k_1 = importWAV('Noise Signals/12k_Ohm1.wav')
time30k, data30k_1 = importWAV('Noise Signals/30k_Ohm1.wav')
time100k, data100k_1 = importWAV('Noise Signals/100k_Ohm1.wav')
time180k, data180k_1 = importWAV('Noise Signals/180k_Ohm1.wav')

time18, data18_2 = importWAV('Noise Signals/18_Ohm2.wav')
time56, data56_2 = importWAV('Noise Signals/56_Ohm2.wav')
time120, data120_2 = importWAV('Noise Signals/120_Ohm2.wav')
time300, data300_2 = importWAV('Noise Signals/300_Ohm2.wav')
time1k, data1k_2 = importWAV('Noise Signals/1k_Ohm2.wav')
time2k, data2k_2 = importWAV('Noise Signals/2k_Ohm2.wav')
time4k, data4k_2 = importWAV('Noise Signals/4k_Ohm2.wav')
time7k, data7k_2 = importWAV('Noise Signals/7k_Ohm2.wav')
time12k, data12k_2 = importWAV('Noise Signals/12k_Ohm2.wav')
time30k, data30k_2 = importWAV('Noise Signals/30k_Ohm2.wav')
time100k, data100k_2 = importWAV('Noise Signals/100k_Ohm2.wav')
time180k, data180k_2 = importWAV('Noise Signals/180k_Ohm2.wav')

v_data18_1L = data18_1['left']
v_data56_1L = data56_1['left']
v_data120_1L = data120_1['left']
v_data300_1L = data300_1['left']
v_data1k_1L = data1k_1['left']
v_data2k_1L = data2k_1['left']
v_data4k_1L = data4k_1['left']
v_data7k_1L = data7k_1['left']
v_data12k_1L = data12k_1['left']
v_data30k_1L = data30k_1['left']
v_data100k_1L = data100k_1['left']
v_data180k_1L = data180k_1['left']

v_data18_1R = data18_1['right']
v_data56_1R = data56_1['right']
v_data120_1R = data120_1['right']
v_data300_1R = data300_1['right']
v_data1k_1R = data1k_1['right']
v_data2k_1R = data2k_1['right']
v_data4k_1R = data4k_1['right']
v_data7k_1R = data7k_1['right']
v_data12k_1R = data12k_1['right']
v_data30k_1R = data30k_1['right']
v_data100k_1R = data100k_1['right']
v_data180k_1R = data180k_1['right']

v_data18_2L = data18_2['left']
v_data56_2L = data56_2['left']
v_data120_2L = data120_2['left']
v_data300_2L = data300_2['left']
v_data1k_2L = data1k_2['left']
v_data2k_2L = data2k_2['left']
v_data4k_2L = data4k_2['left']
v_data7k_2L = data7k_2['left']
v_data12k_2L = data12k_2['left']
v_data30k_2L = data30k_2['left']
v_data100k_2L = data100k_2['left']
v_data180k_2L = data180k_2['left']

v_data18_2R = data18_2['right']
v_data56_2R = data56_2['right']
v_data120_2R = data120_2['right']
v_data300_2R = data300_2['right']
v_data1k_2R = data1k_2['right']
v_data2k_2R = data2k_2['right']
v_data4k_2R = data4k_2['right']
v_data7k_2R = data7k_2['right']
v_data12k_2R = data12k_2['right']
v_data30k_2R = data30k_2['right']
v_data100k_2R = data100k_2['right']
v_data180k_2R = data180k_2['right']

# R values for 1st data collection (old)
# R_18 = 17.962
# R_56 = 56.217
# R_120 = 120.538
# R_300 = 302.203
# R_1k = 990.65
# R_2k = 2411.65
# R_3_9k = 3895.67

# R values for current data set
R_18 = 18.03
R_56 = 55.511
R_120 = 119.855
R_300 = 298.387
R_1k = 983.87
R_2k = 2404.41
R_4k = 3916.84
R_7k = 6.7674E3
R_12k = 11.8645E3
R_30k = 30.019E3
R_100k = 99.52E3
R_180k = 179.7E3


# R_arr = np.array([R_18, R_56, R_120, R_300,
#                   R_1k, R_2k, R_4k, R_7k, R_12k,
#                   R_30k, R_100k, R_180k])

# v_data1_arr = np.array([v_data18_1L, v_data56_1L, v_data120_1L, v_data300_1L, v_data1k_1L, v_data2k_1L, 
#                        v_data4k_1L, v_data7k_1L, v_data12k_1L, v_data30k_1L, v_data100k_1L, v_data180k_1L])

# v_data2_arr = np.array([v_data18_2, v_data56_2, v_data120_2, v_data300_2, v_data1k_2, v_data2k_2,
#                        v_data4k_2, v_data7k_2, v_data12k_2, v_data30k_2, v_data100k_2, v_data180k_2])


R_arr = np.array([R_18, R_56, R_120, R_300,
                  R_1k, R_2k, R_4k, R_7k, R_12k,
                  R_30k])

v_data_1L_arr = np.array([v_data18_1L, v_data56_1L, v_data120_1L, v_data300_1L, v_data1k_1L, v_data2k_1L, 
                       v_data4k_1L, v_data7k_1L, v_data12k_1L, v_data30k_1L], dtype=object)
v_data_1R_arr = np.array([v_data18_1R, v_data56_1R, v_data120_1R, v_data300_1R, v_data1k_1R, v_data2k_1R, 
                       v_data4k_1R, v_data7k_1R, v_data12k_1R, v_data30k_1R], dtype=object)
v_data_2L_arr = np.array([v_data18_2L, v_data56_2L, v_data120_2L, v_data300_2L, v_data1k_2L, v_data2k_2L, 
                       v_data4k_2L, v_data7k_2L, v_data12k_2L, v_data30k_2L], dtype=object)
v_data_2R_arr = np.array([v_data18_2R, v_data56_2R, v_data120_2R, v_data300_2R, v_data1k_2R, v_data2k_2R, 
                       v_data4k_2R, v_data7k_2R, v_data12k_2R, v_data30k_2R], dtype=object)

# fig3 = go.Figure()
# fig3.add_trace(go.Scatter(
#     x=time18, y=data18['left'],
#     name='Left'
# ))
# fig3.add_trace(go.Scatter(
#     x=time18, y=data18['right'],
#     name='Right'
# ))
# fig3.show()

C:\Users\logan\AppData\Local\Temp\ipykernel_25092\2671317029.py:13: WavFileWarning:

Chunk (non-data) not understood, skipping it.



In [86]:
# # Test code for cross correlation

# fs = 48000

# test_data_1 = np.copy(v_data100k_1)
# test_data_2 = np.copy(v_data100k_2)

# f, P_cross = signal.csd(test_data_1, test_data_2, fs, nperseg=1024)
# min_len = np.min([len(test_data_1), len(test_data_2)])

# test_data_1 = test_data_1[:min_len]
# test_data_2 = test_data_2[:min_len]

# f1, P_den1 = signal.welch(test_data_1, fs, nperseg=1024)
# f2, P_den2 = signal.welch(test_data_2, fs, nperseg=1024)

# coherence = np.abs(P_cross)**2 / (P_den1 * P_den2)

# coherence_threshold = 0.99
# correlated_freqs = f[coherence > coherence_threshold]

# filtered_signal_1 = np.copy(v_data300_1)
# filtered_signal_2 = np.copy(v_data300_2)

# for freq in correlated_freqs:
#     # Design a notch filter at the correlated frequency
#     b, a = signal.iirnotch(w0=freq / (fs / 2), Q=30)  # Q factor controls notch width; adjust as needed
#     # Apply the notch filter to both signals
#     filtered_signal_1 = signal.filtfilt(b, a, filtered_signal_1)
#     filtered_signal_2 = signal.filtfilt(b, a, filtered_signal_2)

# ffilt1, P_filt1 = signal.welch(filtered_signal_1, fs, nperseg=1024)
# ffilt2, P_filt2 = signal.welch(filtered_signal_2, fs, nperseg=1024)

# # Plot the original and uncorrelated signals to check the result
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=f, y=np.abs(P_cross), mode='lines', name='Cross-Power Spectral Density |Pxy|'))
# fig.add_trace(go.Scatter(x=f1, y=P_den1, mode='lines', name='Power Spectral Density (Signal 1)'))
# fig.add_trace(go.Scatter(x=f2, y=P_den2, mode='lines', name='Power Spectral Density (Signal 2)'))
# fig.add_trace(go.Scatter(x=f1, y=coherence,  mode='lines', name='coherence'))

# fig.update_layout(
#     title="Cross-Power Spectral Density and Individual Power Spectra",
#     xaxis_title="Frequency (Hz)",
#     yaxis_title="Power",
#     template="plotly_dark"
# )
# fig.show()

In [87]:
# Noise data analysis

Temp = 21.1 + 273.15
Temp_std = 1

fs = 48000

Pden_arr = np.array([])
Pden_std_arr = np.array([])


def G_std_over_G(f, V_in, R_2):
    return np.sqrt( ( (3E-3 + 0.01 * G_piece_func(f) * V_in) / (G_piece_func(f) * V_in) ) ** 2
                                     + (( v_into_amp(3E-3, cR_1, R_2) + 0.01 * V_in) / V_in) ** 2)

fig1 = go.Figure()
fig1.update_layout(
    xaxis_title="Frequency (Hz)",
    yaxis_title="#1 CSD (V^2 / Hz)",
    xaxis_type="log",
    template="plotly_dark"
)

fig2 = go.Figure()
fig2.update_layout(
    xaxis_title="Frequency (Hz)",
    yaxis_title="#2 CSD (V^2 / Hz)",
    xaxis_type="log",
    template="plotly_dark"
)


# Spectral Analysis
r = 0
for v_dat_1L, v_dat_2L, v_dat_1R, v_dat_2R in zip(v_data_1L_arr, v_data_2L_arr, v_data_1R_arr, v_data_2R_arr):
    fc_1, P_den_cross_1 = signal.csd(v_dat_1L, v_dat_1R, fs, nperseg=1024)
    fc_2, P_den_cross_2 = signal.csd(v_dat_2L, v_dat_2R, fs, nperseg=1024)
    fc_1 = fc_1.real
    fc_2 = fc_2.real
    P_den_cross_1 = P_den_cross_1.real
    P_den_cross_2 = P_den_cross_2.real
    P_den1 = P_den_cross_1 / G_piece_func(fc_1) ** 2 / C_interp(fc_1) ** 2
    P_den2 = P_den_cross_2 / G_piece_func(fc_2) ** 2 / C_interp(fc_2) ** 2
    f_flat1 = fc_1[15:400]
    f_flat2 = fc_2[15:400]
    P_den1_flat = P_den1[15:400]
    P_den2_flat = P_den2[15:400]
    
    

    fig1.add_trace(go.Scatter(
        x=fc_1, y=P_den1,
        name='{} Ω'.format(R_arr[r]),
        mode='lines',
        # line=dict(
        #     color=color_arr[r]
        # )
    ))

    fig2.add_trace(go.Scatter(
        x=fc_2, y=P_den2,
        name='{} Ω'.format(R_arr[r]),
        mode='lines',
        # line=dict(
        #     color=color_arr[r]
        # )
    ))

    data_std1 = np.sqrt(
        np.sum(
            ( 2 * P_den1_flat * np.sqrt( (2/40) ** 2 +
                                         (0.2 / G_piece_func(f_flat1) + 0.01) ** 2) ) ** 2
        )
    ) / len(f_flat1)
    data_std2 = np.sqrt(
        np.sum(
            ( 2 * P_den2_flat * np.sqrt( (2/40) ** 2 +
                                         (0.2 / G_piece_func(f_flat2) + 0.01) ** 2) ) ** 2
        )
    ) / len(f_flat2)
    
    Pden_std_arr = np.array([*Pden_std_arr, *[np.sqrt(np.std(P_den1_flat) ** 2 + data_std1 ** 2),
                                            np.sqrt(np.std(P_den2_flat) ** 2 + data_std2 ** 2)]])
    Pden_arr = np.array([*Pden_arr, *[np.mean(P_den1_flat),
                                    np.mean(P_den2_flat)]])
    
    r += 1

# Fitting data
def lin_func(x, a, b):
    return a*x + b

def quad_func(x, a, b, c):
    return a*x**2 + b*x + c

ext_R_arr = np.array([])
for res in R_arr:    
    ext_R_arr = np.array([*ext_R_arr, res, res])

kb_slope_arr = Pden_arr / (4 * Temp)
kb_slope_std_arr = Pden_arr / (4 * Temp) * np.sqrt((Pden_std_arr / Pden_arr) ** 2 + (Temp_std / Temp) ** 2)

params_lin, cov_lin = curve_fit(lin_func, ext_R_arr, kb_slope_arr, sigma=kb_slope_std_arr)
p_lin_err = np.sqrt(np.diag(cov_lin))

params_quad, cov_quad = curve_fit(quad_func, ext_R_arr, kb_slope_arr, sigma=kb_slope_std_arr)
p_quad_err = np.sqrt(np.diag(cov_quad))

R_space = np.linspace(0, max(R_arr), 10000)


# Plotting
fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=ext_R_arr, y=kb_slope_arr,
    error_y=dict(
        type='data',
        array=kb_slope_std_arr,
        color='blue'
    ),
    name='Data', 
    mode='markers',
    marker=dict(
        color='blue'
    )
))

fig3.add_trace(go.Scatter(
    x=R_space, y=lin_func(R_space, *params_lin),
    name='Linear Fit',
    mode='lines',
    line=dict(
        color='red'
    )
))

fig3.add_trace(go.Scatter(
    x=R_space, y=quad_func(R_space, *params_quad),
    name='Quadratic Fit',
    mode='lines',
    line=dict(
        color='green'
    )
))

fig3.update_layout(
    xaxis_title='Resistance (Ω)',
    yaxis_title='<S_f> / 4T (V^2 / (Hz * K))',
    template="plotly_dark"
)

fig1.show()
fig1.write_image('# 1 CSD per R.pdf')
fig1.write_image('# 1 CSD per R.png')
fig2.show()
fig2.write_image('# 2 CSD per R.pdf')
fig2.write_image('# 2 CSD per R.png')
fig3.show()
fig3.write_image('S vs R plot.pdf')
fig3.write_image('S vs R plot.png')


print('Lin param: {} +/- {} , Const param: {} +/- {}'.format(params_lin[0], p_lin_err[0], params_lin[1], p_lin_err[1]))
print('Quad param: {} +/- {} , Lin param: {} +/- {} , Const param: {} +/- {}'.format(params_quad[0], p_quad_err[0], params_quad[1],
                                                                                     p_quad_err[1], params_quad[2], p_quad_err[2]))

C:\Users\logan\AppData\Local\Temp\ipykernel_25092\870586406.py:102: RuntimeWarning:

divide by zero encountered in log10

C:\Users\logan\AppData\Local\Temp\ipykernel_25092\870586406.py:102: RuntimeWarning:

invalid value encountered in cos

C:\Users\logan\AppData\Local\Temp\ipykernel_25092\870586406.py:102: RuntimeWarning:

invalid value encountered in sin



Lin param: 1.4655423124364523e-23 +/- 1.2171037726553788e-24 , Const param: -1.1654345627693684e-21 +/- 5.3416252431462415e-22
Quad param: 1.4609538505789453e-28 +/- 1.4313454885991817e-28 , Lin param: 1.3544727971871258e-23 +/- 1.6315811042279976e-24 , Const param: -9.922311748874635e-22 +/- 5.59878848756772e-22
